# Direct Access versus Getters and Setters

An important consideration when writing a class is the way in which data attributes are accessed from outside an instance. As an example, remember that one of our Drone attributes was altitude.  We would usually get and set its value by writing `.altitude` after the name of the instance:

In [3]:
class Drone:
    
    num_drones = 0
    
    def __init__(self, altitude = 0):
        self.altitude = altitude
        self.ascend_count = 0
        Drone.num_drones += 1
    
    def fly(self):
        print("The drone is flying at", self.altitude, "feet.")
    
    def ascend(self, change):
        self.altitude += change
        self.ascend_count += 1

    
d1 = Drone(100)
print("The Drone's altitude is", d1.altitude)
d1.altitude = 300
print("The Drone's altitude is", d1.altitude)


The Drone's altitude is 100
The Drone's altitude is 300


While that seems natural, it isn't the only style for accessing data attributes.  Some programmers, especially those used to coding in Java, would find it more natural to write a method to return the attribute's value and a method to set the value.  These are known as getter and setter methods.


In [4]:
class Drone:
    
    num_drones = 0
    
    def __init__(self, altitude = 0):
        self.altitude = altitude
        self.ascend_count = 0
        Drone.num_drones += 1
    
    def fly(self):
        print("The drone is flying at", self.altitude, "feet.")
    
    def ascend(self, change):
        self.altitude += change
        self.ascend_count += 1
        
    def get_altitude(self):
        return self.altitude
    
    def set_altitude(self, new_altitude):
        self.altitude = new_altitude

    
d1 = Drone(100)
print("The Drone's altitude is", d1.get_altitude())
d1.set_altitude(300)
print("The Drone's altitude is", d1.get_altitude())

The Drone's altitude is 100
The Drone's altitude is 300


Should you use getter and setter methods?  In this specific example, the answer is probably not.  The getter and setter methods add an extra layer of complexity to our class, without really changing the underlying behavior.  As a matter of encapsulation, we use methods to describe the actions that can be performed on our object, but having a data attribute already suggests that its value can be gotten and set.  Directly accessing the data attribute is more transparent and clarifies that this is a simple operation that only changes one value.  In short, directly accessing data attributes is more Pythonic.

There are many other situations in which getter and setter make sense.  Let's say that we wanted to check to make sure that altitude was never negative.  We could add an if statement to the `set_altitude` method to perform this check.

In [5]:
class Drone:
    
    num_drones = 0
    
    def __init__(self, altitude = 0):
        self.altitude = altitude
        self.ascend_count = 0
        Drone.num_drones += 1
    
    def fly(self):
        print("The drone is flying at", self.altitude, "feet.")
    
    def ascend(self, change):
        self.altitude += change
        self.ascend_count += 1
        
    def get_altitude(self):
        return self.altitude
    
    def set_altitude(self, new_altitude):
        if new_altitude < 0:
            raise Exception("Drone cannot have a negative altitude.")
        self.altitude = new_altitude

    
d1 = Drone(100)
print("The Drone's altitude is", d1.get_altitude())
d1.set_altitude(-10)
print("The Drone's altitude is", d1.get_altitude())

The Drone's altitude is 100


Exception: Drone cannot have a negative altitude.

The setter method is flexible, allowing us to do extra processing when setting an attribute.  As another example, we could use the setter method to increment `ascend_count` whenever the altitude is set to a larger value.

As a futher example, we might want to have a `get_altitude` method, but no `set_altitude` method.  Perhaps we think that altitude should only be changed with a call to `ascend()`.  In this case, we are using our methods as a way of describing the prefered ways of interacting with a Drone object.

## Hidden Attributes

We've argued that there are times in which we want to use getter and setter methods, because we need the flexibility to do extra processing when an attribute is accessed.  Unfortunately, even if we write a perfect setter method, a programmer could always override our wishes by accessing a data attribute directly.  For example, someone could deliberately give our Drone a negative altitude as follows:

In [6]:
class Drone:
    
    num_drones = 0
    
    def __init__(self, altitude = 0):
        self.altitude = altitude
        self.ascend_count = 0
        Drone.num_drones += 1
    
    def fly(self):
        print("The drone is flying at", self.altitude, "feet.")
    
    def ascend(self, change):
        self.altitude += change
        self.ascend_count += 1
        
    def get_altitude(self):
        return self.altitude
    
    def set_altitude(self, new_altitude):
        if new_altitude < 0:
            raise Exception("Drone cannot have a negative altitude.")
        self.altitude = new_altitude

    
d1 = Drone(100)
print("The Drone's altitude is", d1.get_altitude())
d1.altitude = -10
print("The Drone's altitude is", d1.get_altitude())

The Drone's altitude is 100
The Drone's altitude is -10


This violates the spirit of the code we wrote and could result in unexpected errors down the road.  You might be wondering, when this would happen since you know you should never give your drone a negative altitude. You are right; *you* might not. But you can't always predict how your class will be used in the future. How should other people know not to access your attributes in this way?

In other languages, notable Java and C++, the solution is to declare altitude a *private* variable.  This prevents anyone from accessing it from outside the class.  Unlike these languages, however, Python has no private keyword.  The design philosophy of Python dictates that programmers should be warned not to access altitude, but they should still be able to if they really need to.

Python's solution is to make altitude a hidden attribute.  To do this, add 2 underscores to the start of the attribute name (not the end).  The new name will be `__altitude`.  Notice that we can no longer access the attribute directly.

In [12]:
class Drone:
    
    num_drones = 0
    
    def __init__(self, altitude = 0):
        self.__altitude = altitude
        self.ascend_count = 0
        Drone.num_drones += 1
    
    def fly(self):
        print("The drone is flying at", self.__altitude, "feet.")
    
    def ascend(self, change):
        self.__altitude += change
        self.ascend_count += 1
        
    def get_altitude(self):
        return self.__altitude
    
    def set_altitude(self, new_altitude):
        if new_altitude < 0:
            raise Exception("Drone cannot have a negative altitude.")
        self.__altitude = new_altitude

    
d1 = Drone(100)
print("The Drone's altitude is", d1.__altitude)

AttributeError: 'Drone' object has no attribute '__altitude'

The idea here is to code *defensively*. You never know how your classes might be used in the future.  As much as possible, you want to control the ways in which your instances are accessed so that the behaviors are predictable.  When you make an attribute hidden, you're saying that you want to carefully control the ways that it's accessed.  You're encapsulating the hidden complexity of how the class works and presenting a clean interface to other programmers.


## Overriding Hidden Attributes

When you make an attribute hidden in Python, you're essentially saying, "messing with this is not recommended."  But there is no 100% guarrantee that nobody will ever access the attribute directly.  In fact, Python provides a way to get access to hidden attributes from outside a class - it just takes a little extra work.

This is different from languages like C++ and Java. Python's philosophy is that if a programmer really wants to access an attribute, we should trust that they know what they're doing and won't break anything.  According to an popular saying, "we're all consenting adults here."

Most of the time, making an attribute hidden is the end of the story - the situations in which you want to access a hidden attribute from outside a class are very rare.  Still, it's worth seeing how this works so you're not surprised if you see code like this.

To access a hidden attribute, you have to use a special name.  Place an underscore and the name of the class in front of the attribute name.  In this case, `altitude` would become `_Drone__altitude`.  This is known as name mangling.

In [14]:
class Drone:
    
    num_drones = 0
    
    def __init__(self, altitude = 0):
        self.__altitude = altitude
        self.ascend_count = 0
        Drone.num_drones += 1
    
    def fly(self):
        print("The drone is flying at", self.__altitude, "feet.")
    
    def ascend(self, change):
        self.__altitude += change
        self.ascend_count += 1
        
    def get_altitude(self):
        return self.__altitude
    
    def set_altitude(self, new_altitude):
        if new_altitude < 0:
            raise Exception("Drone cannot have a negative altitude.")
        self.__altitude = new_altitude

d1 = Drone(100)
print("The Drone's altitude is", d1._Drone__altitude)
d1._Drone__altitude = 300
print("The Drone's altitude is", d1._Drone__altitude)



The Drone's altitude is 100
The Drone's altitude is 300


As you can see, Python just makes it a little harder to access hidden attributes.  It's an extra step to make sure that nobody does it carelessly.  With some experience working on large coding projects, you may come to appreciate this feature of Python.
